# Model Importing
siebert/sentiment-roberta-large-english

In [7]:
# output in 2023-04-24-04-35-log.txt
!textattack attack --model-from-huggingface siebert/sentiment-roberta-large-english --dataset-from-huggingface rotten_tomatoes --num-examples 15 --parallel --shuffle --log-to-txt

Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/maxwell/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1045.79it/s]
textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Loading pre-trained model from HuggingFace model repository: siebert/sentiment-roberta-large-english
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to text file at path 2023-04-24-04-35-log.txt
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 15
textattack: Worklist candidate size: 8515
  0%|                                                    | 0/15 [00:00<?, ?it/s]Attack

In [12]:
# output in 
!textattack attack --attack-recipe clare --model-from-huggingface siebert/sentiment-roberta-large-english --dataset-from-huggingface rotten_tomatoes --num-examples 15 --parallel --shuffle --log-to-txt 

Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/maxwell/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 813.16it/s]
textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Loading pre-trained model from HuggingFace model repository: siebert/sentiment-roberta-large-english
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to text file at path 2023-04-24-04-51-log.txt
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 15
textattack: Worklist candidate size: 8515
  0%| 

In [7]:
from textattack import Attack
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.goal_functions import UntargetedClassification
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

# We'll use untargeted classification as the goal function.
goal_function = UntargetedClassification(model_wrapper)
# We'll to use our WordSwapEmbedding as the attack transformation.
transformation = WordSwapEmbedding() 
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [3]:
# dataset: An iterable of (text, ground_truth_output) pairs.
#0 means the review is negative
#1 means the review is positive
custom_dataset = [
    ('I hate this movie', 0), #A negative comment, with a negative label
    ('I hate this movie', 1), #A negative comment, with a positive label
    ('I love this movie', 0), #A positive comment, with a negative label
    ('I love this movie', 1), #A positive comment, with a positive label
]
from textattack.datasets import Dataset
dataset_module = Dataset(custom_dataset)

In [8]:
## Full Dataset
import pandas as pd 
from textattack.datasets import Dataset
data = pd.read_csv('Sentiment_Dataset/SemEval17/Clean/semeval.csv')
data['polarity'].replace('POS', 2, inplace=True)
data['polarity'].replace('NEU', 1, inplace=True)
data['polarity'].replace('NEG', 0, inplace=True)

data = data[['text', 'polarity']].values.tolist()
# data = data[:4]+data[5:41]+data[42:100]
data = data[:100]
dataset_module = Dataset(data)

In [9]:
attack.cuda_()

In [4]:
attack.attack("Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making waves. Watch the trailer on Rolling Stone - look...", 1)

In [10]:
results = []
for example, label in dataset_module:
    try:
        result = attack.attack(example, label)
        results.append(result)
    except:
        results.append("Invalid")
    print(len(results))
    print(result.__str__(color_method="ansi"))
results


1
1 (100%) --> 0 (100%)

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making waves. Watch the trailer on Rolling Stone - look...

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making indistinct. Watch the trailer on Rolling Stone - look...
2
1 (100%) --> 0 (100%)

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making waves. Watch the trailer on Rolling Stone - look...

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making indistinct. Watch the trailer on Rolling Stone - look...


In [6]:
results

['Invalid']

In [55]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=10, log_to_csv="log.csv", parallel=True, csv_coloring_style = "file")

attacker = Attacker(attack, dataset_module, attack_args)

attack_results = attacker.attack_dataset()


#The following legacy tutorial code shows how the Attack API works in detail.

# logger = CSVLogger(color_method='html')

# num_successes = 0
# i = 0
# while num_successes < 10:
#     result = next(results_iterable)
#     example, ground_truth_output = dataset[i]
#     i += 1
#     result = attack.attack(example, ground_truth_output)
#     if isinstance(result, SuccessfulAttackResult):
#        logger.log_attack_result(result)
#        num_successes += 1
#        print(f'{num_successes} of 10 successes complete.')

textattack: CSVLogger exiting without calling flush().
textattack: Logging to CSV at path log.csv
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 88
  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:18<02:50, 18.91s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (100%)]]

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making [[waves]]. Watch the trailer on Rolling Stone - look...

Picturehouse's, Pink Floyd's, 'Roger Waters: The Walll - opening 29 Sept is now making [[indistinct]]. Watch the trailer on Rolling Stone - look...


[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:48<03:12, 24.09s/it]

...

[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:  30%|███       | 3/10 [01:22<03:13, 27.59s/it]

--------------------------------------------- Result 3 ---------------------------------------------
[[0 (100%)]] --> [[1 (100%)]]

[[If]] these runway renovations at the airport prevent me from seeing Taylor Swift on Monday, Bad Blood will have a new [[meaning]].

[[Albeit]] these runway renovations at the airport prevent me from seeing Taylor Swift on Monday, Bad Blood will have a new [[sense]].


[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:  40%|████      | 4/10 [01:37<02:25, 24.30s/it]textattack: Exception encountered for input "OrderedDict([('text', 'Excited to read "Go Set a Watchman" on Tuesday.  But can it possibly live up to "To Kill a Mockingbird?"  Any opinions?')])".
textattack: Traceback (most recent call last):
  File "/home/maxwell/Thesis/roberta/venv_txtatck/lib/python3.7/site-packages/textattack/attacker.py", line 591, in attack_from_queue
    result = attack.attack(example, ground_truth_output)
  File "/home/maxwell/Thesis/roberta/venv_txtatck/lib/python3.7/site-packages/textattack/attack.py", line 443, in attack
    example, ground_truth_output
  File "/home/maxwell/Thesis/roberta/venv_txtatck/lib/python3.7/site-packages/textattack/goal_functions/goal_function.py", line 68, in init_attack_example
    result, _ = self.get_result(attacked_text, check_skip=True)
  File "/home/maxwell/Thesis/roberta/venv_txtatck/lib/python3.7/site-packages/textattack/goal_f

--------------------------------------------- Result 4 ---------------------------------------------
[[1 (99%)]] --> [[0 (100%)]]

If you could ask an onstage interview question at [[Miss]] USA tomorrow, what would it be?

If you could ask an onstage interview question at [[Missed]] USA tomorrow, what would it be?


[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:  40%|████      | 4/10 [01:37<02:26, 24.37s/it]


Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/maxwell/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 421.03it/s]
textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Loading pre-trained model from HuggingFace model repository: siebert/sentiment-roberta-large-english
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): RepeatModification
   